In [38]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd

In [66]:
REFORMS = {
    "Baseline": None,
    "Expand refundable CTC": {
        "gov.contrib.congress.wyden_smith.per_child_actc_phase_in": {
            "2024-01-01.2100-12-31": True
        },
        "gov.irs.credits.ctc.refundable.individual_max": {
            "2024-01-01.2100-12-31": 2000
        },
        "gov.irs.credits.ctc.refundable.phase_in.threshold": {
            "2024-01-01.2100-12-31": 0
        }
    },
    "Restore ARPA EITC expansion for childless filers": {
        "gov.irs.credits.eitc.eligibility.age.max": {
            "2023-01-01.2028-12-31": 100
        },
        "gov.irs.credits.eitc.eligibility.age.min": {
            "2023-01-01.2028-12-31": 19
        },
        "gov.irs.credits.eitc.max[0].amount": {
            "2023-01-01.2028-12-31": 1502
        },
        "gov.irs.credits.eitc.phase_in_rate[0].amount": {
            "2023-01-01.2028-12-31": 0.153
        },
        "gov.irs.credits.eitc.phase_out.rate[0].amount": {
            "2023-01-01.2028-12-31": 0.153
        },
        "gov.irs.credits.eitc.phase_out.start[0].amount": {
            "2023-01-01.2028-12-31": 11610
        }
    },
    "SSI Reform": {
        "gov.ssa.ssi.income.exclusions.earned_share": {
            "2024-01-01.2100-12-31": 0.7
        }
    },
    "SNAP Reform": {
        "gov.usda.snap.expected_contribution": {
            "2024-01-01.2100-12-31": 0.2
        },
        "gov.usda.snap.income.deductions.earned_income": {
            "2024-01-01.2100-12-31": 0.3
        },
        "gov.usda.snap.income.limit.gross": {
            "2024-01-01.2100-12-31": 2
        },
        "gov.usda.snap.income.limit.net": {
            "2024-01-01.2100-12-31": 10000
        }
    }
}


In [67]:
def get_earners_above_mtr(reform=None, threshold=0.45, adult_index_limit=3):
    if reform is None:
        sim = Microsimulation(dataset="enhanced_cps_2022")
    else:
        sim = Microsimulation(reform=Reform.from_dict(reform, country_id="us"), dataset="enhanced_cps_2022")
    
    mtr = sim.calculate("marginal_tax_rate", map_to="person", period=2024)

    adult_index = sim.calculate("adult_index", map_to="person", period=2024)

    is_adult = adult_index > 0
    include = is_adult[adult_index <= adult_index_limit]

    total = include.sum()

    exceeds_mtr = include[mtr > threshold]
    return exceeds_mtr.sum() / total
    

In [68]:
results = []

for reform_name, reform_details in REFORMS.items():
    print(f"Running {reform_name}")
    proportion = get_earners_above_mtr(reform=reform_details)
    results.append({"Reform": reform_name, "Proportion Above MTR": proportion})

# Add one more run with all four reforms applied as a tuple.
all_reforms = {k: v for reform in REFORMS.values() if reform is not None for k, v in reform.items()}
results.append({"Reform": "All reforms", "Proportion Above MTR": get_earners_above_mtr(reform=all_reforms)})

Running Baseline
Running Expand refundable CTC
Running Restore ARPA EITC expansion for childless filers
Running SSI Reform
Running SNAP Reform


In [71]:
# Add one more run with all four reforms applied as a tuple.
all_reforms = {k: v for reform in REFORMS.values() if reform is not None for k, v in reform.items()}
results.append({"Reform": "All Reforms", "Proportion Above MTR": get_earners_above_mtr(reform=all_reforms)})

In [74]:
# Create a DataFrame from the results
df = pd.DataFrame(results)

# Add a column for the reform as a percentage of the baseline
df["change"] = (df["Proportion Above MTR"] / df.loc[0, "Proportion Above MTR"]) - 1

# Display the DataFrame
print(df)

                                             Reform  Proportion Above MTR  \
0                                          Baseline              0.105700   
1                             Expand refundable CTC              0.064971   
2  Restore ARPA EITC expansion for childless filers              0.109615   
3                                        SSI Reform              0.081134   
4                                       SNAP Reform              0.083002   
5                                       All reforms              0.079570   

     change  
0  0.000000  
1 -0.385323  
2  0.037038  
3 -0.232416  
4 -0.214738  
5 -0.247211  
